# Importing packets and Stablishing a PATH to the GTFS.zip

In [ ]:
#Import the gtfs-kit module
import gtfs_kit as gk
from pathlib import Path
import pandas as pd
import geopandas as gpd
from geopandas import * 
import shapely
from shapely import *
import matplotlib.pyplot as plt


In [ ]:

#Declare the directory path for the GTFS zip file
path = Path('/home/lubuntu/Téléchargements/TEC-GTFS.zip')

#Read the feed with gtfs-kit
feed = (gk.read_feed(path, dist_units='km'))

#Search for errors and warnings in the feed
feed.validate()

# Line Graphing with GTFS shapes

This notebook intends to present a solution to create the lines of buses whit the points provided by the GTFS datastructure, first we will be working inside a Boundry Box with the following coordinates
    - (3.91256643, 50.46467317),(3.97626359, 50.46490030),(3.97649593, 50.43612858),(3.91283742, 50.43590159)

In [ ]:
coords = [(3.91256643, 50.46467317),(3.97626359, 50.46490030),(3.97649593, 50.43612858),(3.91283742, 50.43590159)]
polygon = shapely.Polygon(coords)
aoi_dict={'geometry':[polygon], 'id':[0]}

In [ ]:
gdf=gpd.GeoDataFrame.from_dict(aoi_dict, geometry='geometry', crs='EPSG:4326')
#gdf.to_file('dataframe.geojson', driver='GeoJSON')
#gdf.to_file('dataframe_pkg.gpkg', driver='GPKG', layer='name') 

# Getting the data

We tell the GTFS file what information we want to get in this case the routes table

In [ ]:
shapes = pd.read_csv('/home/lubuntu/Téléchargements/GTFS/shapes.txt')
trips = pd.read_csv('/home/lubuntu/Téléchargements/GTFS/trips.txt')

Now we create a gdf with all the points from the **GTFS** and then proceed to **filter** the points that are within the bounding box mentioned earlier

In [ ]:
trips

### Creating Dataframes
Now that we have everything we need we create a GeoDataFrame with the id and pointsm also we filter them to just work within the bounding box for processing limitations


In [ ]:
#Creating dataframe
gdf= gpd.GeoDataFrame(shapes,geometry=gpd.points_from_xy(shapes.shape_pt_lon, shapes.shape_pt_lat,crs="EPSG:4326"))
trips_reduced = trips[['shape_id', 'route_id']]

#Applying filter of bounding box
aoi = gdf.geometry.within(polygon)
#Turning the geolist into a usable geodataframe
gdf = gdf[aoi]

In [ ]:
gdf

### Sorting things up
Finally we sort the points by their shape id and in order indicated by **shape_pt_sequence** then we group them making "blocks" by the **shapes_id** and make lines to form every single form. and finally turn the result into a geodataframe called **lines_gdf**

In [ ]:
#Sort the lines by id and Sequence so they are ordered
gdf_sorted = gdf.sort_values(by=['shape_id', 'shape_pt_sequence'])

#Group the points by 'route'
lines = gdf_sorted.groupby('shape_id').apply(lambda x: LineString(x.geometry.tolist()))

#WIP of LINES
#routes = gdf_to_change.groupby('route_id').apply

#Turning the results into a gdf
lines_gdf = gpd.GeoDataFrame(geometry=lines, crs=gdf.crs).reset_index()

Finally we merge the shapes and routes gdf into a final one so we can plot correctly the routes instead of every shape we had

In [ ]:
routes = lines_gdf.merge(trips_reduced, on='shape_id', how='left')

In [ ]:
routes

In [ ]:
routes.plot('route_id')

In [ ]:
routes.explore('route_id')

# Conclusions
Finally this notebook will allow us to work in any bounding box we decide and graph the lines and routes that are within the area, providing us with a geodataframe we can use